In [1]:
import os
import importlib
os.environ['pRT_input_data_path'] = '/net/lem/data2/pRT_input_data'

from retrieval_base.retrieval import RetrievalSetup

# Import input file as 'conf'
conf_string = 'config_fiducial_K_A_new'
conf = importlib.import_module(conf_string)

RS = RetrievalSetup(conf)

species v0.9.0

Working folder: /net/lem/data1/regt/retrieval_base/Luhman_16

Configuration file: /net/lem/data1/regt/retrieval_base/Luhman_16/species_config.ini
Database file: /net/lem/data1/regt/retrieval_base/Luhman_16/species_database.hdf5
Data folder: /net/lem/data1/regt/retrieval_base/Luhman_16/data

Configuration settings:
   - Database: /net/lem/data1/regt/retrieval_base/Luhman_16/species_database.hdf5
   - Data folder: /net/lem/data1/regt/retrieval_base/Luhman_16/data
   - Magnitude of Vega: 0.03

Multiprocessing: mpi4py installed
Process number 1 out of 1...


/net/lem/data1/regt/conda_env/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/net/lem/data1/regt/petitRADTRANS/petitRADTRANS/radtrans.py:119: FutureWarning: pRT_input_data_path was set by an environment variable. In a future update, the path to the petitRADTRANS input_data will be set within a .ini file that will be automatically generated into the user home directory (OS agnostic), inside a .petitradtrans directory
  warnings.warn(f"pRT_input_data_path was set by an environment variable. In a future update, the path to "


  Read line opacities of H2O_pokazatel_main_iso_Sam_new...
  Read line opacities of H2O_181_HotWat78...
  Read line opacities of CO_high_Sam...
  Read line opacities of CO_36_high_Sam...
  Read line opacities of CO_28_high_Sam...
  Read line opacities of CH4_MM_main_iso...
  Read line opacities of NH3_coles_main_iso_Sam...
  Read line opacities of H2S_Sid_main_iso...
  Read line opacities of HF_main_iso_new...

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

  Read line opacities of H2O_pokazatel_main_iso_Sam_new...
  Read line opacities of H2O_181_HotWat78...
  Read line opacities of CO_high_Sam...
  Read line opacities of CO_36_high_Sam...
  Read line opacities of CO_28_high_Sam...
  Read line opacities of CH4_MM_main_iso...
  Read line opacities of NH3_coles_main_iso_Sam...
  Read line opacities of H2S_Sid_main_iso...
  Read line opacities of HF_main_iso_new...

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

  Read line opacitie

In [ ]:
RS.model.Rot['K2166_B']

array([0.05235988, 0.05235988, 0.05235988, ..., 1.51843645, 1.51843645,
       1.51843645])

In [5]:
#RS.model.Chem['K2166_B'].test_parameter
RS.model.Cloud['K2166_B'](RS.ParamTable, RS.model.Chem['K2166_B'])

?


In [6]:
RS.model.Chem['K2166_B'].test_parameter

'test'

In [3]:
import numpy as np
cube = np.random.uniform(0, 1, RS.ParamTable.table['idx_free'].max()+1)
phys_cube = RS.ParamTable(cube)
RS.ParamTable.table

,name,m_set,Param,val,idx_free
0,parallax,all,None,496,None
1,inclination,all,None,18,None
2,distance,all,None,2.016129,None
3,log_a,all,<retrieval_base.parameters_new.Parameter objec...,-0.386795,0
4,a,all,None,0.410397,None
5,log_l,all,<retrieval_base.parameters_new.Parameter objec...,-2.432873,1
6,l,all,None,0.003691,None
7,R_p,all,<retrieval_base.parameters_new.Parameter objec...,0.952275,2
8,rv,all,<retrieval_base.parameters_new.Parameter objec...,28.804612,3
9,vsini,all,<retrieval_base.parameters_new.Parameter objec...,23.693089,4


In [4]:
m_set = 'K2166_B'
RS.ParamTable.queried_m_set = ['all', m_set]

RS.model.PT[m_set](RS.ParamTable)
RS.model.Chem[m_set](RS.ParamTable, RS.model.PT[m_set].temperature)
RS.model.Chem[m_set].VMRs

{'12CO': array([0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00258474, 0.00258474, 0.00258474, 0.00258474,
        0.00258474, 0.00257932, 0.00257482, 0.00256465, 0.00255141,
        0.00253213, 0.0025015 , 0.00245277, 0.00238416, 0.00228284]),
 'H2O': array([0.00156818, 0.00156818, 0.00156818, 0.00156818, 0.00156818,
        0.00156818, 0.00156818, 0.00156818, 0.00156818, 0.00156818,
        0.00156818, 0.00156818, 0.00156818, 0.00156818, 0.00156818,
        0.00156818, 0.00156818, 0.00156818, 0.00156818, 0.00156818,
        0.00156818, 0.00156818,

In [ ]:
RS.ParamTable.queried_m_set = ['K2166_B','all']
temperature = RS.model.PT['K2166_B'](RS.ParamTable)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,7))
plt.plot(temperature, RS.model.PT['K2166_B'].pressure, 'k')
plt.yscale('log')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
RS.ParamTable.table

In [ ]:
import numpy as np
cube = np.random.uniform(0, 1, 26)
phys_cube = RS.ParamTable(cube)
RS.ParamTable.table

In [ ]:
%timeit phys_cube = RS.ParamTable(cube)

In [ ]:
RS.ParamTable.add_param(name='log_g', m_set='all', val=4.5)

In [ ]:
RS.ParamTable.table.loc[RS.ParamTable.table['name']=='g', 'val']

In [ ]:
(RS.ParamTable.table['name']=='g').any()

In [ ]:
RS.ParamTable.table.loc[RS.ParamTable.table['name']=='g', 'val'] = 9.6

In [ ]:
RS.ParamTable.table.loc['g','val'] = 9.6

In [ ]:
queried_m_set = 'all'
name = 'M_p'

query = f'm_set=="{queried_m_set}" and name=="{name}"'
RS.ParamTable.table.query(query)

In [ ]:
RS.ParamTable.table

In [ ]:
RS.ParamTable.get('opa_base_gray_0')

In [ ]:
from parameters_new import ParameterTable

In [ ]:
constant_params = {
    'const_1': 1., 
    'const_2': 2., 
    'A': {'const_3': 3.},  
    }
free_params = {
    'free_4': ('U', (0,6), r'free_a'), 
    'free_5': ('G', (5,1), r'free_b'), 
    'A': {
        'free_6': ('U', (5,7), r'free_6')
        },
    'B': {
        'free_7': ('U', (4,7), r'free_7'), 
        'log_free_8': ('U', (2,3), r'log_free_8')
        },
    }

PI = ParameterTable(
    free_params, constant_params, 
    model_settings=['A','B'], 
    PT_kwargs={
        'PT_mode':'free_gradient', 
        'A':{'n_T_knots':5, 'PT_mode':'grid'}, 
        #'A':{'n_T_knots':5}
        }, 
    )

PI(cube=[0.5, 0.4, 0.5, 0.5, 0.5], ndim=5)
PI.table

In [ ]:
PI.PT_kwargs

In [ ]:
cloud_kwargs = {
    #'cloud_mode': 'gray', 
    'wave_cloud_0': 2.0, 
    'A': {'wave_cloud_0': 2.5}, 
    'B': {'cloud_mode': 'EddySed'}, 
    }
new_cloud_kwargs = PI.expand_per_model_setting(cloud_kwargs, main_m_set='all')
print(new_cloud_kwargs)

new_cloud_kwargs['A'].update(new_cloud_kwargs['all'])
print(new_cloud_kwargs)

In [ ]:
PI.queried_m_set = 'B'; print(PI.get('free_8'))
PI.queried_m_set = ''; print(PI.get('free_8'))

PI.queried_m_set = 'all'; print(PI.get('free_5'))